In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.gold.dim_shows")

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS workspace.gold.dim_shows (
  show_id BIGINT,
  show_name STRING,
  show_type STRING,
  show_language STRING,
  show_status STRING,
  show_genres STRING,
  show_premiered STRING,
  show_rating_average DOUBLE,
  show_webchannel_name STRING,
  show_network_name STRING
)
""")



In [0]:
import pandas as pd

# Columnas finales que quieres en la dimensión
columns = [
    "show_id",
    "show_name",
    "show_type",
    "show_language",
    "show_status",
    "show_genres",
    "show_premiered",
    "show_rating_average",
    "show_webchannel_name",
    "show_network_name",
]

# Leer silver en pandas
df = spark.table("workspace.silver.tvmaze").toPandas()

# Seleccionar columnas (si falta alguna, se crea con NaN), limpiar y ordenar
dim = (
    df.reindex(columns=columns)
      .dropna(subset=["show_id"])
      .drop_duplicates(subset=["show_id"])
      .sort_values("show_name", ignore_index=True, na_position="last")
)

df_spark = spark.createDataFrame(dim)


In [0]:

# Usamos overwrite para reemplazar completamente los datos existentes
df_spark.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("workspace.gold.dim_shows")